In [1]:
from pprint import pprint
import urllib
from bs4 import BeautifulSoup
from webdriver_manager.firefox import GeckoDriverManager
import time
import sys
import datetime

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# original_file_descriptor = sys.stdout

In [3]:
# sys.stdout = open("./scrape_log.txt", "a")

In [4]:
print("------------- STARTING ON " + str(datetime.datetime.now()) + " -------------")

------------- STARTING ON 2019-06-19 15:16:48.271523 -------------


In [5]:
# Given a URL string, opens the URL in a headless Firefox instance and returns a reference to the webdriver object

def obtain_driver(url):
    buttons = []
    driver_options = Options()
#     driver_options.headless = True
    
    print("\n\n*****" + str(datetime.datetime.now()) + "*****")
    
    driver = webdriver.Firefox(options=driver_options, executable_path=GeckoDriverManager().install())
    # driver.install_addon("~/.mozilla/firefox/f4t6w0s5.default-release/extensions/uBlock0@raymondhill.net.xpi", temporary=True)

    driver.get(url)
    
    time.sleep(1)
    
    return driver

In [19]:
# Given a DOI, uses obtain_driver to open up the search site and search the DOI to open up the paper in the page. Returns the webdriver associated with it.

def search_paper(doi, driver):
    if "www.researchgate.net/search" in driver.current_url:
        search_input = "search-form__input"
        search_button = "search-form__left-button"
    else:
        search_input = "lite-page__header-search-input"
        search_button = "lite-page__header-search-button"
    
    search_field = driver.find_elements_by_class_name(search_input)[0]
    search_field.send_keys(doi)
    
#     time.sleep(1)
    
    enter_field = driver.find_elements_by_class_name(search_button)[0]
    driver.execute_script("arguments[0].click()", enter_field)
    
    time.sleep(1)
    
    if "www.researchgate.net/search" in driver.current_url:
        print("Paper not found (search)")
#         driver.quit()
        return None
    else: 
        return driver

In [8]:
# Given the webdriver, shows more authors. Returns the modified webdriver.

def show_authors(driver): 
    
    time.sleep(1)
    
    if not driver:
        print("Exiting: No driver")
        return 0
    
    first_start = time.time()
    
    buttons = driver.find_elements_by_class_name("show-more-authors")
    
    print("show_more_authors button: " + str(buttons))
    
    # Need to address the condition of the button not existing (i.e. <= 4 authors for paper)
    
    if len(buttons) == 1:
        time.sleep(3)
        print("click")
        driver.execute_script("arguments[0].click();", buttons[0])
#         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "show-less-authors")))
        time.sleep(3)    
    
    first_end = time.time()

    print("DONE - " + str(first_end-first_start))
    
    soup_it(driver)
    
    return 1
    
# driver.quit()


In [23]:
# Given the webdriver, parses its source for author URLs.

def soup_it(driver):
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))
    
#     driver.quit()

In [14]:
driver = obtain_driver("https://www.researchgate.net/publication/284825989_Evaluation_of_carbon_dioxide-nitrogen_separation_through_fixed_bed_measurements_and_simulations")
search_paper(3, driver)

# driver.quit()



*****2019-06-19 15:18:20.048087*****

Checking for linux64 geckodriver:v0.24.0 in cache
Driver found in /home/local/NIST/jfl2/.wdm/geckodriver/v0.24.0/linux64/geckodriver
https://www.researchgate.net/search?context=publicSearchHeader&q=3
Paper not found (search)


In [24]:
# Testing on an example

driver = obtain_driver("https://www.researchgate.net/search")
time.sleep(1)

doi_list = ["10.1021/acs.iecr.5b03509", "10.1007/s10450-014-9639-3", "10.1002/adfm.200500561", "10.1002/adfm.200500563"]
# source = search_paper("10.1021/acs.iecr.5b03509", driver)   # 6 authors
# source = search_paper("10.1007/s10450-014-9639-3", driver)  # 9 authors
# source = search_paper("10.1002/adfm.200500561", driver)     # 4 authors
# source = search_paper("10.1002/adfm.200500563")     # doesn't exist

for doi in doi_list:
    show_authors(search_paper(doi, driver))
    time.sleep(2)

# show_authors(source)

# if exit_val != 0:
# searched = show_authors(source)
# soup_it(searched)



*****2019-06-19 15:47:36.986473*****

Checking for linux64 geckodriver:v0.24.0 in cache
Driver found in /home/local/NIST/jfl2/.wdm/geckodriver/v0.24.0/linux64/geckodriver
show_more_authors button: [<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="46cb92aa-ff48-4cb5-a73a-0269f35937aa", element="0e9480fb-51e2-4a0c-be2b-6a18891a070d")>]
click
DONE - 7.529669761657715
https://www.researchgate.net/profile/Juliana_Coelho5
https://www.researchgate.net/profile/Ana_Ribeiro25
https://www.researchgate.net/profile/Alexandre_Ferreira4
https://www.researchgate.net/scientific-contributions/2082644774_Sebastiao_MP_Lucena
https://www.researchgate.net/profile/Alirio_Rodrigues
https://www.researchgate.net/profile/Diana_Azevedo
show_more_authors button: [<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="46cb92aa-ff48-4cb5-a73a-0269f35937aa", element="bb4dc121-cd1c-4706-bb00-f67f32cf72f6")>]
click
DONE - 8.414184808731079
https://www.researchgate.net/profile/Rafael_Rios

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=49249): Max retries exceeded with url: /session/628dd697-ebac-404f-b4d9-efaa7726f93c/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f599f67edd8>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [10]:
# Obtains author URLs from a basic request. Can't interact with the webpage to show more authors if there are > 4 authors.

def obtain_author_researchgate(website_url):
    request = urllib.request.Request(
        url = website_url,
        data = None,
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0'
        }
    )

    testurl = urllib.request.urlopen(request)
    soup = BeautifulSoup(testurl, 'html.parser')

    thing = soup.find_all('div', {"class": "nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-e-text--clamp nova-v-person-list-item__title"})
    for stuff in thing:
        print(stuff.find('a').get("href"))